In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.1 MB 11.4 MB/s 
     |████████████████████████████████| 59 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 40.0 MB/s 
     |████████████████████████████████| 596 kB 39.8 MB/s 
     |████████████████████████████████| 895 kB 38.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 12.9 MB/s 
     |████████████████████████████████| 243 kB 40.5 MB/s 
     |████████████████████████████████| 1.1 MB 27.3 MB/s 
     |████████████████████████████████| 132 kB 51.1 MB/s 
     |████████████████████████████████| 192 kB 45.6 MB/s 
     |████████████████████████████████| 160 kB 49.1 MB/s 
     |████████████████████████████████| 271 kB 46.9 MB/s 


In [ ]:
### imports 


# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy
# Models

from transformers import AutoTokenizer

# Training

import tensorflow

# Evaluation

# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns

from transformers import AutoModelForSequenceClassification


from datasets import load_dataset

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_metric


import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import plot_precision_recall_curve


checkpoint = "bert-base-uncased"



In [ ]:
### configs
### preproccess the raw data into title text combinations
### preprocess data 
raw_datasets = load_dataset('csv', data_files={ "train": '/content/data/train.csv', "test": '/content/data/test.csv', "valid": '/content/data/valid.csv'})
### tokenize data
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["titletext"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Using custom data configuration default-add5f1d6bbf0b340


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-add5f1d6bbf0b340/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
training_args = TrainingArguments("test-trainer")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['BodyMarkdown', 'OpenStatus', 'OwnerCreationDate', 'OwnerUndeletedAnswerCountAtPostTime', 'OwnerUserId', 'PostClosedDate', 'PostCreationDate', 'PostId', 'ReputationAtPostCreation', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'Title', 'attention_mask', 'input_ids', 'labels', 'text', 'title', 'titletext', 'token_type_ids'],
        num_rows: 798
    })
    test: Dataset({
        features: ['BodyMarkdown', 'OpenStatus', 'OwnerCreationDate', 'OwnerUndeletedAnswerCountAtPostTime', 'OwnerUserId', 'PostClosedDate', 'PostCreationDate', 'PostId', 'ReputationAtPostCreation', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'Title', 'attention_mask', 'input_ids', 'labels', 'text', 'title', 'titletext', 'token_type_ids'],
        num_rows: 9001
    })
    valid: Dataset({
        features: ['BodyMarkdown', 'OpenStatus', 'OwnerCreationDate', 'OwnerUndeletedAnswerCountAtPostTime', 'OwnerUserId', 'PostClosedDate', 'PostCreationDate', 'PostId', 'ReputationAtPo

In [ ]:
columns_to_return = ['input_ids', 'labels', 'attention_mask']
tokenized_datasets.set_format(type='torch', columns=columns_to_return)

In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = numpy.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)

In [ ]:
# Fine tune the model 
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: PostCreationDate, Tag5, Tag3, title, Title, OwnerUndeletedAnswerCountAtPostTime, BodyMarkdown, Tag2, PostClosedDate, text, OpenStatus, titletext, OwnerUserId, Tag4, Tag1, ReputationAtPostCreation, OwnerCreationDate, PostId.
***** Running training *****
  Num examples = 798
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 300


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=0.03762611389160156, metrics={'train_runtime': 429.1221, 'train_samples_per_second': 5.579, 'train_steps_per_second': 0.699, 'total_flos': 629887866531840.0, 'train_loss': 0.03762611389160156, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: PostCreationDate, Tag5, Tag3, title, Title, OwnerUndeletedAnswerCountAtPostTime, BodyMarkdown, Tag2, PostClosedDate, text, OpenStatus, titletext, OwnerUserId, Tag4, Tag1, ReputationAtPostCreation, OwnerCreationDate, PostId.
***** Running Prediction *****
  Num examples = 9001
  Batch size = 8


Step,Training Loss


NameError: ignored

In [ ]:
preds = numpy.argmax(predictions.predictions, axis=-1)
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [ ]:

metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

holdoutData = tokenized_datasets["valid"]

# get label prediction
labelPrediction = preds

# use prediction to get metrics
precision, recall, thresholds = precision_recall_curve(predictions.label_ids, labelPrediction)
average_precision = average_precision_score(predictions.label_ids, labelPrediction)


In [ ]:
average_precision

0.9751243781094527